In [1]:
import pandas as pd
import numpy as np

from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
from numpy import array

## importing libraries

import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import time
from sklearn import preprocessing
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import sklearn.metrics
import math
from numpy import mean
from numpy import std

from sklearn.metrics import mean_squared_error, r2_score


print('Done importing libraries')

Done importing libraries


In [2]:
df2 = pd.read_csv('data/allMonsoon_delhi.csv')
df2.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
df2.drop(["index"], axis=1, inplace=True)



Ozone = df2['O3']

OzoneP24 = Ozone.shift(-24)
OzoneP24 = OzoneP24.replace(np.nan, Ozone.mean())
df2['O3P24'] = OzoneP24

df2.info()
print(df2.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   a            14640 non-null  int64  
 1   City         14640 non-null  object 
 2   Date         14640 non-null  object 
 3   Time         14640 non-null  object 
 4   PM2.5        14640 non-null  float64
 5   PM10         14640 non-null  float64
 6   NO           14640 non-null  float64
 7   NO2          14640 non-null  float64
 8   NOx          14640 non-null  float64
 9   NH3          14640 non-null  float64
 10  CO           14640 non-null  float64
 11  SO2          14640 non-null  float64
 12  O3           14640 non-null  float64
 13  Benzene      14640 non-null  float64
 14  Toluene      14640 non-null  float64
 15  Xylene       14640 non-null  float64
 16  O3P24        14640 non-null  float64
 17  Temperature  14640 non-null  float64
dtypes: float64(14), int64(1), object(3)
memory usa

In [3]:
## defining variables

#X = df2[['PM10', 'NO', 'NO2', 'CO', 'SO2', 'O3', 'Toluene', 'Temp']] # , 'Humid', Xylene
X = df2[['PM10', 'NO', 'NO2', 'NH3','CO', 'SO2', 'O3', 'Toluene', 'Xylene', 'Temperature']]

y = df2['O3P24']

from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train, y_test = train_test_split(X,y,random_state=0, test_size=0.1)

print("Size of training dataset: {} rows".format(X_train1.shape[0]))
print("Size of testing dataset: {} rows".format(X_test1.shape[0]))

print('Done defining variables')

Size of training dataset: 13176 rows
Size of testing dataset: 1464 rows
Done defining variables


In [4]:
## scaling variables

'''importing libraries'''
from sklearn.preprocessing import RobustScaler

scaler = preprocessing.RobustScaler()
X_train_scaled1 = scaler.fit_transform(X_train1)
X_test_scaled1 = scaler.fit_transform(X_test1)

print('Done scaling :)')

Done scaling :)


In [5]:
## linear regression
t0 = time.time()

'''importing library'''
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold

'''create linear regressor object'''
lnRegressor = LinearRegression()

'''fitting regressor'''
lnRegressor.fit(X_train1, y_train)

# cv stuff

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

cvln_r2scores = cross_val_score(lnRegressor, X_train1, y_train, cv = 10, scoring = 'r2')
print(cvln_r2scores)
cvln_rmsescores = cross_val_score(lnRegressor, X_train1, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvln_rmsescores)
cvln_maescores = cross_val_score(lnRegressor, X_train1, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvln_maescores)

# calculating r^2 adj score

cvln_adj = []

n = len(X_train1)
k = len(X_train1.columns)
for r in cvln_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvln_adj.append(adj_r2)    

cvln_r2score = round((np.mean(cvln_r2scores)), 4)
cvln_adjscore = round((np.mean(cvln_adj)), 4)
cvln_rmsescore = (round(abs(np.mean(cvln_rmsescores)),2))
cvln_maescore = (round(abs(np.mean(cvln_maescores)),2))


print(cvln_r2score, cvln_adjscore, cvln_rmsescore, cvln_maescore)

t1 = time.time()
total = t1-t0

print("done training linear regressor | time taken: %f seconds" %total)
yPredln = lnRegressor.predict(X_test1)
print('The average R2 value for linear regressor is :', cvln_r2score)

[0.46419866 0.42245043 0.40611536 0.47920801 0.44657588 0.3859502
 0.46129985 0.51740514 0.42037896 0.4315057 ]
[-18.22413383 -19.673194   -21.109792   -18.11004286 -17.70399137
 -21.17852703 -17.66970289 -16.92844876 -18.54379753 -20.14205822]
[-11.48428658 -12.2371847  -12.37468732 -11.8042819  -11.98264511
 -12.58856011 -11.84691385 -11.78737625 -12.16828865 -12.70674009]
0.4435 0.4431 18.93 12.1
done training linear regressor | time taken: 3.343044 seconds
The average R2 value for linear regressor is : 0.4435


In [6]:
## kneighbors regression
print("STARTING: KN")

t0 = time.time()

'''importing library'''
from sklearn.neighbors import KNeighborsRegressor

'''create regressor object'''
knRegressor = KNeighborsRegressor(n_neighbors = 4, metric = 'minkowski', p = 1)

'''fitting regressor'''
knRegressor.fit(X_train_scaled1, y_train)

# cv stuff

cvkn_r2scores = cross_val_score(knRegressor, X_train1, y_train, cv = 10, scoring = 'r2')
print(cvkn_r2scores)
cvkn_rmsescores = cross_val_score(knRegressor, X_train1, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvkn_rmsescores)
cvkn_maescores = cross_val_score(knRegressor, X_train1, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvkn_maescores)

# calculating r^2 adj score

cvkn_adj = []

n = len(X_train1)
k = len(X_train1.columns)
for r in cvkn_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvkn_adj.append(adj_r2)    

cvkn_r2score = round((np.mean(cvkn_r2scores)), 4)
cvkn_adjscore = round((np.mean(cvkn_adj)), 4)
cvkn_rmsescore = (round(abs(np.mean(cvkn_rmsescores)),2))
cvkn_maescore = (round(abs(np.mean(cvkn_maescores)),2))

print(cvkn_r2score, cvkn_adjscore, cvkn_rmsescore, cvkn_maescore)


t1 = time.time()
total = t1-t0

print("done training knn regressor | time taken: %f seconds" %total)

yPredkn = knRegressor.predict(X_test_scaled1)
print('The average R2 value for KNN Regressor is :', cvln_r2score)

STARTING: KN
[0.56768858 0.52104965 0.56256915 0.52736153 0.55397916 0.55195195
 0.60239986 0.57925368 0.5211995  0.52152036]
[-16.36979918 -17.91535795 -18.11705535 -17.25249321 -15.89350644
 -18.09073663 -15.1802513  -15.80650975 -16.85403543 -18.47874693]
[ -9.74117423 -10.60944565 -10.25657124 -10.15846556  -9.74654134
 -10.24969765  -9.80210135  -9.9291419  -10.27819088 -10.62540422]
0.5509 0.5506 17.0 10.14
done training knn regressor | time taken: 9.276652 seconds
The average R2 value for KNN Regressor is : 0.4435


In [7]:
## svr regressor
print("STARTING: SVR")

t0 = time.time()

'''importing library'''
from sklearn.svm import SVR

'''create regressor object'''
svrRegressor = SVR(C = 10, gamma = 0.1, kernel= 'rbf')

'''fitting regressor'''
svrRegressor.fit(X_train_scaled1, y_train)

# cv stuff

cvsvr_r2scores = cross_val_score(svrRegressor, X_train_scaled1, y_train, cv = 10, scoring = 'r2')
print(cvsvr_r2scores)
cvsvr_rmsescores = cross_val_score(svrRegressor, X_train1, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvsvr_rmsescores)
cvsvr_maescores = cross_val_score(svrRegressor, X_train1, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvsvr_maescores)


# calculating r^2 adj score

cvsvr_adj = []

n = len(X_train1)
k = len(X_train1.columns)
for r in cvsvr_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvsvr_adj.append(adj_r2)

cvsvr_r2score = round((np.mean(cvsvr_r2scores)), 4)
cvsvr_adjscore = round((np.mean(cvsvr_adj)), 4)
cvsvr_rmsescore = (round(abs(np.mean(cvsvr_rmsescores)),2))
cvsvr_maescore = (round(abs(np.mean(cvsvr_maescores)),2))

print(cvsvr_r2score, cvsvr_adjscore, cvsvr_rmsescore, cvsvr_maescore)

t1 = time.time()
total = t1-t0

print("done training svr regressor | time taken: %f seconds" %total)

yPredsvr = svrRegressor.predict(X_test_scaled1)
print('The average R2 value for SVR Regressor is :', cvsvr_r2score)

STARTING: SVR
[0.56318243 0.52439117 0.50912784 0.5341731  0.54433393 0.50288351
 0.58541283 0.60250399 0.52258395 0.52841857]
[-24.62086992 -25.68323245 -27.29322268 -24.87421664 -23.27559958
 -26.87856686 -23.77991436 -24.23828135 -23.9869839  -26.57767782]
[-17.71316644 -17.97810078 -18.84738979 -17.67153499 -17.27034083
 -18.59369199 -17.94344822 -18.26447412 -17.1030971  -18.72009741]
0.5417 0.5414 25.12 18.01
done training svr regressor | time taken: 1273.578159 seconds
The average R2 value for SVR Regressor is : 0.5417


In [8]:
## defining variables WITH MULTICOLLINEARITY: random forest, decision tree, maybe adaboost, and xgboost

X = df2[['PM10', 'NO', 'NO2', 'NOx', 'NH3','CO', 'SO2', 'O3', 'Toluene', 'Xylene', 'Temperature']]

y = df2['O3P24']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0, test_size=0.1)

print("Size of training dataset: {} rows".format(X_train.shape[0]))
print("Size of testing dataset: {} rows".format(X_test.shape[0]))

print('Done defining NEW variables')

Size of training dataset: 13176 rows
Size of testing dataset: 1464 rows
Done defining NEW variables


In [9]:
## random forest regressor
print("STARTING: rf")
t0 = time.time()

'''importing library'''
from sklearn.ensemble import RandomForestRegressor

'''create regressor object'''
rfRegressor = RandomForestRegressor(max_depth=50, random_state=0, n_estimators=250) 

'''fitting regressor'''
rfRegressor.fit(X_train, y_train)

# cv stuff

cvrf_r2scores = cross_val_score(rfRegressor, X_train, y_train, cv = 10, scoring = 'r2') # x_train was scaled for some reason, check if it changes things?
print(cvrf_r2scores)
cvrf_rmsescores = cross_val_score(rfRegressor, X_train, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvrf_rmsescores)
cvrf_maescores = cross_val_score(rfRegressor, X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvrf_maescores)


# calculating r^2 adj score

cvrf_adj = []

n = len(X_train)
k = len(X_train.columns)
for r in cvrf_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvrf_adj.append(adj_r2)

cvrf_r2score = round((np.mean(cvrf_r2scores)), 4)
cvrf_adjscore = round((np.mean(cvrf_adj)), 4)
cvrf_rmsescore = (round(abs(np.mean(cvrf_rmsescores)),2))
cvrf_maescore = (round(abs(np.mean(cvrf_maescores)),2))

print(cvrf_r2score, cvrf_adjscore, cvrf_rmsescore, cvrf_maescore)

print(cvrf_r2score, cvrf_adjscore)

t1 = time.time()
total = t1-t0

print("done training random forest regressor | time taken: %f seconds" %total)
yPredrf = rfRegressor.predict(X_test)
print('The average R2 value for Random Forest Regressor is :', cvrf_r2score)

## printing feature importances

importances = rfRegressor.feature_importances_

sorted_indices = np.argsort(importances)[::-1]

print('feature importances:::')
print(sorted_indices)

STARTING: rf
[0.61483059 0.57755103 0.6037719  0.59496949 0.60080434 0.54414097
 0.65719783 0.6536489  0.57228585 0.5853817 ]
[-15.45150869 -16.8254764  -17.24271066 -15.97096641 -15.03609474
 -18.2477463  -14.09540249 -14.3411542  -15.92954658 -17.2014486 ]
[ -9.11831498 -10.04921218  -9.93994643  -9.71579192  -9.5300137
 -10.12673799  -9.18789794  -9.29585148  -9.80111188 -10.21518042]
0.6005 0.6001 16.03 9.7
0.6005 0.6001
done training random forest regressor | time taken: 1300.127202 seconds
The average R2 value for Random Forest Regressor is : 0.6005
feature importances:::
[ 7 10  8  0  4  6  2  1  3  5  9]


In [10]:
## decision tree regressor
print('STARTING: dt')
t0 = time.time()

'''importing library'''
from sklearn.tree import DecisionTreeRegressor

'''create regressor object'''
dtRegressor = DecisionTreeRegressor(random_state=0, max_depth = 6)

'''fitting regressor'''
dtRegressor.fit(X_train,y_train)

# cv stuff

cvdt_r2scores = cross_val_score(dtRegressor, X_train, y_train, cv = 10, scoring = 'r2')
print(cvdt_r2scores)
cvdt_rmsescores = cross_val_score(dtRegressor, X_train, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvdt_rmsescores)
cvdt_maescores = cross_val_score(dtRegressor, X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvdt_maescores)


# calculating r^2 adj score

cvdt_adj = []

n = len(X_train)
k = len(X_train.columns)
for r in cvdt_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvdt_adj.append(adj_r2)

cvdt_r2score = round((np.mean(cvdt_r2scores)), 4)
cvdt_adjscore = round((np.mean(cvdt_adj)), 4)
cvdt_rmsescore = (round(abs(np.mean(cvdt_rmsescores)),2))
cvdt_maescore = (round(abs(np.mean(cvdt_maescores)),2))

print(cvdt_r2score, cvdt_adjscore, cvdt_rmsescore, cvdt_maescore)

t1 = time.time()
total = t1-t0

print("done training decision tree regressor | time taken: %f seconds" %total)

yPreddt = dtRegressor.predict(X_test)
print('The average R2 value for Decision Tree Regressor is :', cvdt_r2score)

STARTING: dt
[0.51119009 0.44857188 0.42946945 0.51261931 0.4943122  0.44864149
 0.54072037 0.53770222 0.4560828  0.48632501]
[-17.40664203 -19.22315627 -20.69056541 -17.51949118 -16.92323502
 -20.06831893 -16.31527091 -16.56863464 -17.96358457 -19.14630615]
[-10.52647266 -11.42161864 -11.71708145 -11.01053301 -11.05388313
 -11.65224088 -11.01718457 -10.99198914 -11.41190651 -11.7728252 ]
0.4866 0.4861 18.18 11.26
done training decision tree regressor | time taken: 2.758951 seconds
The average R2 value for Decision Tree Regressor is : 0.4866


In [11]:
## adaboost regressor
print('STARTING: adaboost regressor')
t0 = time.time()

'''importing library'''
from sklearn.ensemble import AdaBoostRegressor

'''create regressor object'''
adaRegressor = AdaBoostRegressor(random_state=0, learning_rate = 0.1, n_estimators=100)

'''fitting regressor'''
adaRegressor.fit(X_train, y_train)

# cv stuff

cvada_r2scores = cross_val_score(adaRegressor, X_train, y_train, cv = 10, scoring = 'r2')
print(cvada_r2scores)
cvada_rmsescores = cross_val_score(adaRegressor, X_train, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvada_rmsescores)
cvada_maescores = cross_val_score(adaRegressor, X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvada_maescores)

# calculating r^2 adj score

cvada_adj = []

n = len(X_train)
k = len(X_train.columns)
for r in cvada_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvada_adj.append(adj_r2)

cvada_r2score = round((np.mean(cvada_r2scores)), 4)
cvada_adjscore = round((np.mean(cvada_adj)), 4)
cvada_rmsescore = (round(abs(np.mean(cvada_rmsescores)),2))
cvada_maescore = (round(abs(np.mean(cvada_maescores)),2))

print(cvada_r2score, cvada_adjscore, cvada_rmsescore, cvada_maescore)

t1 = time.time()
total = t1-t0

print("done training Adaboost regressor | time taken: %f seconds" %total)

yPredada = adaRegressor.predict(X_test)
print('The average R2 value for Adaboost Regressor is :', cvada_r2score)

STARTING: adaboost regressor
[0.41136811 0.41601579 0.43112264 0.4028609  0.32418344 0.34865892
 0.40813252 0.43947077 0.38013909 0.36358324]
[-19.10147441 -19.7824827  -20.66056672 -19.39211077 -19.56395076
 -21.81213561 -18.52114953 -18.24420523 -19.17669371 -21.311381  ]
[-12.70574486 -13.08821484 -13.1698798  -13.16540738 -13.96995327
 -14.17274468 -13.24386308 -12.90894418 -13.36961118 -14.28571983]
0.3926 0.392 19.76 13.41
done training Adaboost regressor | time taken: 112.433918 seconds
The average R2 value for Adaboost Regressor is : 0.3926


In [12]:
## xgboost regressor
print('STARTING: xgboost regressor')
t0 = time.time()

'''importing library'''
import xgboost as xgb

'''create regressor object'''
xgbRegressor = xgb.XGBRegressor(learning_rate=0.1, max_depth=10, n_estimators=300, verbosity = 0, random_state = 0, silent = True)

'''fitting regressor'''
xgbRegressor.fit(X_train, y_train)

# cv stuff

cvxgb_r2scores = cross_val_score(xgbRegressor, X_train, y_train, cv = 10, scoring = 'r2')
print(cvxgb_r2scores)
cvxgb_rmsescores = cross_val_score(xgbRegressor, X_train, y_train, cv = 10, scoring = 'neg_root_mean_squared_error')
print(cvxgb_rmsescores)
cvxgb_maescores = cross_val_score(xgbRegressor, X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error')
print(cvxgb_maescores)


# calculating r^2 adj score

cvxgb_adj = []

n = len(X_train)
k = len(X_train.columns)
for r in cvxgb_r2scores:
    adj_r2 = 1-(((1-r)*(n-1))/(n-k-1))
    cvxgb_adj.append(adj_r2)

cvxgb_r2score = round((np.mean(cvxgb_r2scores)), 4)
cvxgb_adjscore = round((np.mean(cvxgb_adj)), 4)
cvxgb_rmsescore = (round(abs(np.mean(cvxgb_rmsescores)),2))
cvxgb_maescore = (round(abs(np.mean(cvxgb_maescores)),2))

print(cvxgb_r2score, cvxgb_adjscore, cvxgb_rmsescore, cvxgb_maescore)

t1 = time.time()
total = t1-t0

print("done training xgboost regressor | time taken: %f seconds" %total)

yPredxgb = xgbRegressor.predict(X_test)
print('The average R2 value for xgboost Regressor is :', cvxgb_r2score)

STARTING: xgboost regressor
[0.62480877 0.58571338 0.63245467 0.57992627 0.60578817 0.56455078
 0.63149186 0.63697686 0.59300984 0.55428154]
[-15.25005248 -16.66213679 -16.60689199 -16.26485109 -14.94193948
 -17.83457298 -14.6143412  -14.68226205 -15.53883872 -17.83491721]
[ -9.05497683  -9.92527098  -9.91460906  -9.97049556  -9.47741449
  -9.92180235  -9.33622563  -9.47068509  -9.74872718 -10.42290764]
0.6009 0.6006 16.02 9.72
done training xgboost regressor | time taken: 374.460297 seconds
The average R2 value for xgboost Regressor is : 0.6009


In [13]:
##### printing cross-validations scores
# printing cross-validaion r^2 scores

print('The R^2 value for Linear Regression is         :', cvln_r2score)
print('The R^2 value for KNN Regressor is             :', cvkn_r2score)
print('The R^2 value for SVM Regressor is             :', cvsvr_r2score)
print('The R^2 value for Random Forests Regressor is  :', cvrf_r2score)
print('The R^2 value for Decision Tree Regressor is   :', cvdt_r2score)
print('The R^2 value for AdaBoost Regressor is        :', cvada_r2score)
print('The R^2 value for XGBoost Regressor is         :', cvxgb_r2score)

# printing cross-validation adjusted r^2 scores

print('The Adj. R^2 value for Linear Regression is        :', cvln_adjscore)
print('The Adj. R^2 value for KNN Regressor is            :', cvkn_adjscore)
print('The Adj. R^2 value for SVM Regressor is            :', cvsvr_adjscore)
print('The Adj. R^2 value for Random Forests Regressor is :', cvrf_adjscore)
print('The Adj. R^2 value for Decision Tree Regressor is  :', cvdt_adjscore)
print('The Adj. R^2 value for AdaBoost Regressor is       :', cvada_adjscore)
print('The Adj. R^2 value for XGBoost Regressor is        :', cvxgb_adjscore)

# printing cross-validation rmse scores

print('The RSME value for Linear Regression is         :', cvln_rmsescore)
print('The RSME value for KNN Regressor is             :', cvkn_rmsescore)
print('The RSME value for SVM Regressor is             :', cvsvr_rmsescore)
print('The RSME value for Random Forests Regressor is  :', cvrf_rmsescore)
print('The RSME value for Decision Tree Regressor is   :', cvdt_rmsescore)
print('The RSME value for AdaBoost Regressor is        :', cvada_rmsescore)
print('The RSME value for XGBoost Regressor is         :', cvxgb_rmsescore)

# printing cross-validation mae scores

print('The MAE value for Linear Regression is        :', cvln_maescore)
print('The MAE value for KNN Regressor is            :', cvkn_maescore)
print('The MAE value for SVM Regressor is            :', cvsvr_maescore)
print('The MAE value for Random Forests Regressor is :', cvrf_maescore)
print('The MAE value for Decision Tree Regressor is  :', cvdt_maescore)
print('The MAE value for AdaBoost Regressor is       :', cvada_maescore)
print('The MAE value for XGBoost Regressor is        :', cvxgb_maescore)

The R^2 value for Linear Regression is         : 0.4435
The R^2 value for KNN Regressor is             : 0.5509
The R^2 value for SVM Regressor is             : 0.5417
The R^2 value for Random Forests Regressor is  : 0.6005
The R^2 value for Decision Tree Regressor is   : 0.4866
The R^2 value for AdaBoost Regressor is        : 0.3926
The R^2 value for XGBoost Regressor is         : 0.6009
The Adj. R^2 value for Linear Regression is        : 0.4431
The Adj. R^2 value for KNN Regressor is            : 0.5506
The Adj. R^2 value for SVM Regressor is            : 0.5414
The Adj. R^2 value for Random Forests Regressor is : 0.6001
The Adj. R^2 value for Decision Tree Regressor is  : 0.4861
The Adj. R^2 value for AdaBoost Regressor is       : 0.392
The Adj. R^2 value for XGBoost Regressor is        : 0.6006
The RSME value for Linear Regression is         : 18.93
The RSME value for KNN Regressor is             : 17.0
The RSME value for SVM Regressor is             : 25.12
The RSME value for Ran